In [38]:
import numpy as np
import pymc as pm
from pytensor.compile.ops import as_op
import pytensor.tensor as pt
import arviz as az
import matplotlib.pyplot as plt

points=100 #=Samples
from scipy.fft import fft, fftfreq


#MAKE SOME REAL DATA
#@as_op(itypes=[pt.dscalar, pt.dscalar, pt.dvector], otypes=[pt.dvector])
def find_dominant_frequencies(data, sampling_rate=100):

    n_waves, n_samples = data.shape
    
    # Compute the FFT for each row
    fft_results = np.abs(fft(data, axis=1))
    
    # Create frequency axis
    frequencies = fftfreq(n_samples, 1 / sampling_rate)
    
    # We only need the positive frequencies
    positive_freq_mask = frequencies > 0
    positive_frequencies = frequencies[positive_freq_mask]
    fft_results = fft_results[:, positive_freq_mask]
    
    # Find the dominant frequency for each sine wave
    dominant_frequencies = positive_frequencies[np.argmax(fft_results, axis=1)]
    
    return dominant_frequencies


#@as_op(itypes=[pt.dscalar, pt.dscalar, pt.dvector], otypes=[pt.dvector])
def make_sine(amp, freq, phase, offset, points=100, noise=0):
    x = np.linspace(0, 10, points)
    sine = amp * np.sin(2 * np.pi * freq * x + phase) + offset
    if noise > 0:
        sine += np.random.normal(0, noise, size=len(sine))
    return sine


points = 100  # Samples per sine wave
n_waves = 20  # Reduced number of waves

# Generate observed data
true_amp, true_freq, true_phase, true_offset = 1.0, 1.0, 0.0, 0.5
observed_sines = np.array([make_sine(true_amp, true_freq, true_phase, true_offset, noise=0.1) for _ in range(n_waves)])
observed_freqs = find_dominant_frequencies(observed_sines)

In [39]:
@as_op(itypes=[pt.dscalar, pt.dscalar, pt.dvector], otypes=[pt.dvector])
def find_dominant_frequencies(data, sampling_rate=100):

    n_waves, n_samples = data.shape
    
    # Compute the FFT for each row
    fft_results = np.abs(fft(data, axis=1))
    
    # Create frequency axis
    frequencies = fftfreq(n_samples, 1 / sampling_rate)
    
    # We only need the positive frequencies
    positive_freq_mask = frequencies > 0
    positive_frequencies = frequencies[positive_freq_mask]
    fft_results = fft_results[:, positive_freq_mask]
    
    # Find the dominant frequency for each sine wave
    dominant_frequencies = positive_frequencies[np.argmax(fft_results, axis=1)]
    
    return dominant_frequencies


@as_op(itypes=[pt.dscalar, pt.dscalar, pt.dvector], otypes=[pt.dvector])
def make_sine(amp, freq, phase, offset, points=100, noise=0):
    x = np.linspace(0, 10, points)
    sine = amp * np.sin(2 * np.pi * freq * x + phase) + offset
    if noise > 0:
        sine += np.random.normal(0, noise, size=len(sine))
    return sine

In [40]:
import numpy as np
from scipy.fft import fft, fftfreq
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import pytensor
import pytensor.tensor as pt

# Define the PyMC model
with pm.Model() as model:
    # Priors
    amplitude = pm.Uniform("amplitude", lower=0, upper=10)
    frequency = pm.Uniform("frequency", lower=0, upper=5)
    phase = pm.Uniform("phase", lower=-np.pi, upper=np.pi)
    offset = pm.Normal("offset", mu=0, sigma=2)
    sigma = pm.HalfNormal("sigma", sigma=1)

    # Define x outside the Deterministic
    x = np.linspace(0, 10, points)

    # Use pm.Deterministic to create the wave
    wave = [make_sine(amplitude, frequency, phase, offset) for _ in range(n_waves)]
    
    # Use the custom FFT in a Deterministic
    modeled_freq = find_dominant_frequencies(wave, sampling_rate=100)

    Y_obs = pm.Normal("Y_obs", 
                      mu=modeled_freq, 
                      sigma=sigma, 
                      observed=observed_freqs)

    # Inference with reduced number of samples
    trace = pm.sample(500, tune=500, return_inferencedata=True)

# Plot the results
az.plot_trace(trace)
plt.tight_layout()
plt.show()

# Print the summary of the posterior
print(az.summary(trace, var_names=["amplitude", "frequency", "phase", "offset", "sigma", "modeled_freq"]))

# Print the true parameters
print(f"True parameters: amplitude={true_amp}, frequency={true_freq}, phase={true_phase}, offset={true_offset}")

ValueError: We expected 3 inputs but got 4.

In [18]:
find_dominant_frequencies(observed_sines)

array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10.])

In [22]:
wave.shape

Shape.0

In [23]:
type(wave)

pytensor.tensor.variable.TensorVariable

In [32]:
with model:
    wave_test = wave.eval({amplitude: 1.0, frequency: 1.0, phase: 0.0, offset: 0.5})
    freq_test = find_dominant_frequencies(wave)
    print(f"Test wave shape: {wave_test.shape}")
    print(f"First few values of test wave: {wave_test[:5]}")
fft_results = np.abs(fft(wave_test))
print("res",freq_test)

ValueError: not enough values to unpack (expected 2, got 1)

In [14]:
amplitude.eval().item()

9.022329404230423